This notebook is the next step after scrapping posts and comments from the subreddit r/AITA and analysing the data collected & trying applying our first Text Classification: <br />
https://github.com/Nico404/scrap_reddit <br />
https://github.com/Nico404/AITA_data_exploration_and_ML/blob/master/AITA_data_exploration.ipynb <br /> 
https://github.com/Nico404/AITA_data_exploration_and_ML/blob/master/Zero_shot_model.ipynb

In [2]:
import pandas as pd
import pickle
from transformers import pipeline
verbose=0

In [29]:
# Load the data from AITA_data_exploration.ipynb
with open('data/pickled_post_data.pkl', 'rb') as f:
    data = pickle.load(f)

posts_df = pd.DataFrame(data)


posts_df["verdict"] = posts_df["comment_results"].apply(lambda x: max(x, key=x.get))

posts_df = posts_df.drop("comment_results", axis=1)
posts_df

# let's keep it at 3K rows since it's the AutoTrain limit for free usage 
posts_df.to_csv("data/AITA_labeled_posts.csv", index=False)
posts_df



,post_id,post_content,post_title,verdict
0,10uxee0,"I know this post sounds super petty, but this ...",AITA for telling my boyfriend I'll shave my le...,NTA
1,10ur722,My daughter Bryn F9 is going on a trip to a ne...,AITA for pulling my daughter from a waterpark ...,NTA
2,10upxdd,Alright so my son (17) has weekly therapy appo...,AITA for not letting an elderly woman have my ...,NTA
3,10v2vra,We live three blocks away from my parents and ...,AITA for taking my kids to my parents house to...,NTA
4,10ung90,My daughter (16) and I have gotten into a mass...,AITA for calling my daughter a selfish insecur...,NTA
...,...,...,...,...
3502,1128lsq,"So I am 21F, my friend ""Sasha"" is 20F and her ...",AITA for telling my friend I don't like her si...,YTA
3503,1123c1c,"On Saturday, me and one of my best friends wer...",AITA for rolling my eyes at my friend?,NTA
3504,1122vfk,"(English is not my native language) So, a bit ...",AITA for not signing my daughter in for a ches...,YTA
3505,1122rza,"Derek (25M, fake name obvs) is my (25M) best f...",AITA for calling my friend's gf a jealous b wo...,YTA


Let's upload our dataset to Hugging Face
https://huggingface.co/datasets/nicoco404/AITA_labeled_posts

Now let's create a new autotrain project - Text Classification (Multi-class)
https://ui.autotrain.huggingface.co/new

We'll use <b>post_content</b> as text input, and <b>verdict</b> as the target and launch the AutoTrain. 
Free evaluation will run and compare 5 models on 3K lines

In [33]:
result_data = {'Model ID': ['#3535895495 putrid-cormorant', ' #3535895498 idiotic-lyrebird', '#3535895497 supportive-chamois', '#3535895496 ethical-peafowl', '#3535895499 juicy-weasel'],
        'Loss': [0.7609, 0.7623, 0.7666, 0.7741, 0.7805],
        'Accuracy': [0.7628, 0.7628, 0.7628, 0.7628, 0.7628],
        'Precision macro': [0.1092, 0.1805, 0.1090, 0.1090, 0.1090],
        'Precision micro': [0.7628, 0.7628, 0.7628, 0.7628, 0.7628],
        'Precision weighted': [0.5829, 0.6687, 0.5819, 0.5819, 0.5819],
        'Recall macro': [0.1429, 0.1440, 0.1429, 0.1429, 0.1429],
        'Recall micro': [0.7628, 0.7628, 0.7628, 0.7628, 0.7628],
        'Recall weighted': [0.7628, 0.7628, 0.7628, 0.7628, 0.7628],
        'F1 macro': [0.1238, 0.1264, 0.1236, 0.1236, 0.1236],
        'F1 micro': [0.7628, 0.7628, 0.7628, 0.7628, 0.7628],
        'F1 weighted': [0.6608, 0.6633, 0.6602, 0.6602, 0.6602]
       }

df = pd.DataFrame(result_data)
df

,Model ID,Loss,Accuracy,Precision macro,Precision micro,Precision weighted,Recall macro,Recall micro,Recall weighted,F1 macro,F1 micro,F1 weighted
0,#3535895495 putrid-cormorant,0.7609,0.7628,0.1092,0.7628,0.5829,0.1429,0.7628,0.7628,0.1238,0.7628,0.6608
1,#3535895498 idiotic-lyrebird,0.7623,0.7628,0.1805,0.7628,0.6687,0.1440,0.7628,0.7628,0.1264,0.7628,0.6633
2,#3535895497 supportive-chamois,0.7666,0.7628,0.1090,0.7628,0.5819,0.1429,0.7628,0.7628,0.1236,0.7628,0.6602
3,#3535895496 ethical-peafowl,0.7741,0.7628,0.1090,0.7628,0.5819,0.1429,0.7628,0.7628,0.1236,0.7628,0.6602
4,#3535895499 juicy-weasel,0.7805,0.7628,0.1090,0.7628,0.5819,0.1429,0.7628,0.7628,0.1236,0.7628,0.6602


### Definition time to pick the best model 

Accuracy is a performance metric used to evaluate the performance of a classification model. It represents the percentage of correctly classified instances out of the total instances in the dataset. In other words, accuracy measures how often the model is correct in its predictions. It is calculated as the ratio of the number of correctly classified instances to the total number of instances in the dataset.

Loss: Loss value indicates how well the model is performing during training. It is a numerical representation of the difference between the predicted outputs and the actual outputs. The lower the loss value, the better the model is at making accurate predictions.

Precision: Precision measures how many of the predicted positive labels are actually correct. It is the proportion of true positives (TP) over the sum of true positives and false positives (FP). In other words, it answers the question: "Out of all the predicted positive cases, how many are actually positive?" A high precision score means that the model has a low false positive rate.

Recall: Recall measures how many of the actual positive cases were correctly identified by the model. It is the proportion of true positives over the sum of true positives and false negatives (FN). In other words, it answers the question: "Out of all the actual positive cases, how many did the model correctly identify as positive?" A high recall score means that the model has a low false negative rate.

F1 score: F1 score is the harmonic mean of precision and recall. It combines both measures and provides a more balanced evaluation of the model's performance. The F1 score is calculated as 2 * (precision * recall) / (precision + recall). It ranges from 0 to 1, where 1 represents the best possible performance and 0 represents the worst.



### Soooo, I am clearly not an expert but something seems fishy to me. 
<pre> _
<_><
</pre>
- Accuracy of all 5 models is the exact same ? 
- Models seem competent enough though
- Precision and Recall are also identical thus F1 is the same
- let's then pick the model with the lowest Loss value, which would be the charming <b> putrid-cormorant </b>
    
    https://huggingface.co/nicoco404/autotrain-aita-post-classifier-3535895495

While I'm fetching fresh data to test this putrid model, let's launch the training of models on <i>Post Titles</i> since they are much shorter than <i>Post Contents</i>

In [41]:
# New post that has 100% NTA verdict
prompt = """"recently, my friends and i went to a restaurant for dinner, where i noticed there was a cute waiter that was serving us. i told my friends, not thinking much of it. throughout the whole dinner, they proceeded to embarrass me about it, sometimes around the waiter, even asking to take a selfie with him. at the end of the dinner, while waiting for a friend to use the restroom, i noticed a friend of mine talking to the waiter 1v1, and i panicked, worried they'd embarrass me again but worse. i left (the bill was already paid for atp) and waited outside for them. they all come out smiling and laughing, saying they told the waiter i thought he was cute, and they got his number. i felt beyond embarrassed now. they put his number in my phone, and i said fuck it and sent a text. i wasn't expecting much to come out of this, so i was shocked to see him reply. we continued to talk throughout the night, even after my friends and i parted ways. the next morning, i was happy, the waiter and i were having a good conversation the same morning. later at lunch, my friend confessed that it wasnt the waiter that i was texting, it was my friend. my friend group was in on it, and even made a groupchat to laugh at me behind my back, leaking the messages and such. i was so mortified, they told me in front of most of our friend group (as i went to dinner with only 4 of them) i angrily left, sending a message to our discord server saying "fuck you guys" and went off on them. they thought i was upset at being catfished, and while i am somewhat upset, i was mainly upset for being turned into a joke and laughed at behind my back by people who i called friends. 2 of them were my best friends. some of them had said im overreacting, and some say im making it a bigger problem than it needs to be. my family says i should drop them, but idk what to do. some people say im right for being upset, and im not overreacting."""

# Loading access token for API
with open('access_token', 'r') as f:
    access_token = f.read()


In [43]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("nicoco404/autotrain-aita-post-classifier-3535895495", use_auth_token=access_token)
tokenizer = AutoTokenizer.from_pretrained("nicoco404/autotrain-aita-post-classifier-3535895495", use_auth_token=access_token)

inputs = tokenizer(prompt, return_tensors="pt")

outputs = model(**inputs)


In [56]:
import torch
from torch.nn.functional import softmax

# Get the logits tensor
outputs = model(**inputs)
logits = outputs.logits

# Map label indices to label names found in model config file
# https://huggingface.co/nicoco404/autotrain-aita-post-classifier-3535895495/blob/main/config.json
label_map = {
    0: "ESH",
    1: "INFO",
    2: "NAH",
    3: "NTA",
    4: "YTA",
    5: "YWBTA",
    6: "YWNBTA"
}

# Apply softmax function to the logits so we get probability values
probs = softmax(logits, dim=1)

# Sort the labels and scores by descending score
sorted_labels = sorted(label_map.values(), key=lambda x: probs[0][list(label_map.values()).index(x)].item(), reverse=True)
sorted_scores = [f"{probs[0][list(label_map.values()).index(label)].item() * 100:.2f}%" for label in sorted_labels]

# Print the sorted labels and scores
for label, score in zip(sorted_labels, sorted_scores):
    print(f"{label}: {score}")
    

NTA: 81.39%
YTA: 11.77%
ESH: 3.23%
NAH: 1.66%
INFO: 1.08%
YWBTA: 0.48%
YWNBTA: 0.39%


## That's so good !

Before we try running more test data on our model, let's get back to our second batch of models trained on titles. Results are almost identical: 
Different input, same dataset ? 

In [47]:
data_title_model = {
    'Model ID': ['each-porpoise', 'insubstantial-bear', 'fussy-nightingale', 'incompatible-chimpanzee', 'chief-fish'],
    'Loss': [0.7624, 0.7675, 0.7740, 0.7593, 0.7836],
    'Accuracy': [0.7628, 0.7628, 0.7628, 0.7628, 0.7628],
    'Precision macro': [0.1090, 0.1090, 0.1090, 0.1090, 0.1090],
    'Precision micro': [0.7628, 0.7628, 0.7628, 0.7628, 0.7628],
    'Precision weighted': [0.5819, 0.5819, 0.5819, 0.5819, 0.5819],
    'Recall macro': [0.1429, 0.1429, 0.1429, 0.1429, 0.1429],
    'Recall micro': [0.7628, 0.7628, 0.7628, 0.7628, 0.7628],
    'Recall weighted': [0.7628, 0.7628, 0.7628, 0.7628, 0.7628],
    'F1 macro': [0.1236, 0.1236, 0.1236, 0.1236, 0.1236],
    'F1 micro': [0.7628, 0.7628, 0.7628, 0.7628, 0.7628],
    'F1 weighted': [0.6602, 0.6602, 0.6602, 0.6602, 0.6602],
}

df = pd.DataFrame(data_title_model)
df


,Model ID,Loss,Accuracy,Precision macro,Precision micro,Precision weighted,Recall macro,Recall micro,Recall weighted,F1 macro,F1 micro,F1 weighted
0,each-porpoise,0.7624,0.7628,0.109,0.7628,0.5819,0.1429,0.7628,0.7628,0.1236,0.7628,0.6602
1,insubstantial-bear,0.7675,0.7628,0.109,0.7628,0.5819,0.1429,0.7628,0.7628,0.1236,0.7628,0.6602
2,fussy-nightingale,0.7740,0.7628,0.109,0.7628,0.5819,0.1429,0.7628,0.7628,0.1236,0.7628,0.6602
3,incompatible-chimpanzee,0.7593,0.7628,0.109,0.7628,0.5819,0.1429,0.7628,0.7628,0.1236,0.7628,0.6602
4,chief-fish,0.7836,0.7628,0.109,0.7628,0.5819,0.1429,0.7628,0.7628,0.1236,0.7628,0.6602
